In [100]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [112]:
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y),n_col))
    for i in range(len(Y)):
        binarized[i,Y[i]] = 1
    return binarized

def load_data(path, filename, y_index=-1, split_ratio=0.7):
    data = pd.read_csv(os.path.join(path,filename),index_col=0).sample(frac=1).reset_index(drop=True)
    names = list(data.iloc[:,-1].unique())
    for i in range(len(names)):
        data = data.replace(names[i],i)
    x, y = pd.DataFrame(data.iloc[:,:-1]).astype('float32'), np.reshape(data.values[:,-1].astype('int32'), (-1,1))
    print(x.shape, y.shape)
    split = int(split_ratio * x.shape[0])
    x_train, x_test, y_train, y_test = x[:split], x[split:], to_one_hot(y[:split]), to_one_hot(y[split:])
    return ((x_train, y_train),(x_test, y_test))

def model_fn(features, labels, mode, params):
    
    # Defining the Network
    net = tf.feature_column.input_layer(features, params['inp_data'])
    for unit in params['hidden_units']:
        net = tf.layers.dense(net, unit, activation=tf.nn.relu, kernel_initializer=tf.initializers.random_normal)
    out = tf.layers.dense(net, params['out_units'], activation=tf.nn.softmax)
    
    # if predict mode, return the predictions of the model
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {'pred_val' : out}
        return tf.estimator.EstimatorSpec(mode, predictions)
    
    # Calculate loss and metrics
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=out)
    mae = tf.metrics.accuracy(labels=labels, predictions=out, name='acc_op')
    metrics = {'Accuracy' : mae}
    tf.summary.scalar('Accuracy', mae[1])
    
    # if evaluation mode, return the metric and loss
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    
    # If neither then it has to be training mode
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, train_op=train_op, loss=loss)
    
def train_inp_fn(inp_data, out_data, batch_size):
    
    data = tf.data.Dataset.from_tensor_slices((dict(inp_data),out_data))
    data = data.shuffle(1000).repeat().batch(batch_size)
    
    return data.make_one_shot_iterator().get_next()

def eval_inp_fn(inp_data, out_data, batch_size):
    
    data = tf.data.Dataset.from_tensor_slices((dict(inp_data),out_data))
    data = data.batch(batch_size)
    
    return data.make_one_shot_iterator().get_next()

In [113]:
((train_x, train_y),(test_x,test_y)) = load_data("","../Iris.csv")

(150, 4) (150, 1)


In [114]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((105, 4), (105, 3), (45, 4), (45, 3))

In [115]:
my_feature_cols = []
for i in train_x.columns:
    my_feature_cols.append(tf.feature_column.numeric_column(i))

In [116]:
classifier = tf.estimator.Estimator(model_fn=model_fn,params={'inp_data': my_feature_cols,'hidden_units': [8, 16, 16, 16],'out_units': 3})
classifier.train(input_fn=lambda:train_inp_fn(train_x, train_y, 32), steps=10000)
eval_result = classifier.evaluate(input_fn=lambda:eval_inp_fn(test_x, test_y, 32))
print('\nTest set Accuracy: {Accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2_kh378q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7a08410d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa

INFO:tensorflow:global_step/sec: 880.234
INFO:tensorflow:loss = 0.86394477, step = 7101 (0.114 sec)
INFO:tensorflow:global_step/sec: 799.051
INFO:tensorflow:loss = 0.86394614, step = 7201 (0.125 sec)
INFO:tensorflow:global_step/sec: 808.551
INFO:tensorflow:loss = 0.89519644, step = 7301 (0.124 sec)
INFO:tensorflow:global_step/sec: 771.528
INFO:tensorflow:loss = 0.73894477, step = 7401 (0.130 sec)
INFO:tensorflow:global_step/sec: 898.937
INFO:tensorflow:loss = 1.0201948, step = 7501 (0.111 sec)
INFO:tensorflow:global_step/sec: 940.098
INFO:tensorflow:loss = 0.9576961, step = 7601 (0.106 sec)
INFO:tensorflow:global_step/sec: 832.489
INFO:tensorflow:loss = 0.8014448, step = 7701 (0.120 sec)
INFO:tensorflow:global_step/sec: 680.028
INFO:tensorflow:loss = 0.8326951, step = 7801 (0.150 sec)
INFO:tensorflow:global_step/sec: 729.574
INFO:tensorflow:loss = 0.8326964, step = 7901 (0.135 sec)
INFO:tensorflow:global_step/sec: 836.642
INFO:tensorflow:loss = 0.8951949, step = 8001 (0.118 sec)
INFO:t

In [110]:
pred_res = classifier.predict(input_fn=lambda:eval_inp_fn(test_x, test_y,32))

In [111]:
l = []
for p in pred_res:
    l.extend(p['pred_val'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpfppnn8jr/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
